<a href="https://colab.research.google.com/github/KinuthiaWambugu/colabnotebooksAA/blob/AA/Workshop_4_1_Etudiant_EN_REV_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Combinatorial optimization by metaheuristic <br/> Workshop</center>

![logoEI_1_6.jpg](attachment:logoEI_1_6.jpg)

We meet up with our student, whose Smartphone is bursting at the seams. He has not given up taking his music library with him, but his attempt to rely on the [knapsack](https://en.wikipedia.org/wiki/Knapsack_problem) problem "*Given a set of items, each with a weight and a value, determine which items to include in the collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.* was not very conclusive.
The approach by linear programming did not work, due to the integer variables, which make its problem-solving by the simplex impossible. It's a shame, because this algorithmic problem corresponds to many challenges, especially from logistics. Which ones can you imagine&nbsp;?

<em></em>

1.   Selection of investments and portfolios
2.   Resource allocation
3. Inventory management
4. Cargo loading
5. Generating keys for the Merkle-Hellman and other knapsack cryptosystems.



# 1 Basic modelling and implementation

We will start by setting up a simple generic approach to solve this knapsack problem.


## 1.1 Heuristic modelling

The [_Hill Climbing_](https://en.wikipedia.org/wiki/Hill_climbing) algorithm (that we should sometimes rather call _Valley Descending_) is a fairly simple solution to implement. What is this algorithm&nbsp;?

<font color="yellow">
<em>This is a popular heuristic search method that adapts to optimization problems and uses local search to identify the optimum. <br>
Hill climbing is a popular heuristic search technique used for solving optimization problems, including the knapsack problem, where we aim to find a solution that maximizes or minimizes a certain objective function under specific constraints. It’s a local search algorithm that iteratively improves a solution by moving in the direction of higher "value" or "fitness" until it reaches a peak (a solution that’s better than all its neighbors).</em>
</font>


The first (and main) step in modelling a problem for a metaheuristic is therefore the definition of the neighbourhood structure (at least in the case of a trajectory method). What simple modification of a solution can we consider in our case of knapsack problem&nbsp;?

<font color ="yellow">
<em>Removing an item currently in the knapsack is another way to modify the solution. This reduces the total weight and might make room for higher-value items in subsequent moves.</em>
</font>

What’s left to address is the computational representation of our data. Knowing that a solution represents, for each object, the fact that we take this object or not, how will we computationally model such a solution&nbsp;?

<font color = "yellow">
<em>Using a boolean structure with T or F</em>

From there, representing the weights and values of each object is quite natural. What structures will be adapted&nbsp;?

<font color = "yellow">
<em>With 2 boolean arrays for weights and another for values of objects.</em>

## 1.2 Basic structures and functions


You already know the very general principle of the algorithms that we are going to implement. It's about generating valid solutions, assessing their value, and keeping the best one. Which Python data structures are we going to use to prioritize runtime performance&nbsp;? Propose a structure for&nbsp;:
* an instance of the problem
* a solution of one instance

<font color = "yellow">
<em>
Instance - no. of objects we're working on, defined with their weights and values and the bags capacity. {Dictionary - best for searching - 0(1)}

Solution - tuple of booleans.
</em>
</font>

Let's start with instances. If we want to test our functions, we must be able to randomly generate an instance (given a weight and a maximum value for any object). For the number of objects and the bag’s capacity, two integers will do&nbsp;:

In [45]:
nb_objets = 10
capacite = 20

With Python’s standard [random](https://docs.python.org/fr/3/library/random.html) library, our random instance generation function will be very simple to write. It will be able to return a tuple of dictionaries `(weight, value)`, which it will have generated with Comprehension.

In [55]:
import random

def random_objets(poids_max, val_max):
 """
 This function generates objects of random weight and value
 (limited by the values passed in parameter).

 Returns a tuple of 2 dictionaries (weight,value)
 """

 poids_objets = {i:random.randint(1, poids_max) for i in range(nb_objets)}
 valeur_objets = {i:random.randint(1, val_max) for i in range(nb_objets)}

 return poids_objets, valeur_objets

random.seed(a=3) # using a seed explicitly will reproduce the initial conditions
 # and compare the behaviour of different algorithms on the same instance

poids_objets, valeur_objets = random_objets(10, 10)
print(str(poids_objets) + "\n" + str(valeur_objets))

{0: 4, 1: 10, 2: 9, 3: 3, 4: 6}
{0: 10, 1: 8, 2: 10, 3: 2, 4: 10}


Now that we can generate a random instance, we will be able to implement and test the basic functions that any metaheuristic needs. Which ones&nbsp;?

<font color = "yellow">
<em>Function to calculate the weight of the solution, i.e., sum of the wweights of the objects in a bag<br>
Function calculating the value of a solution, so that the metaheuristic can compare the 2 solutions.
</em>
</font>

Therefore, what must be done is to successfully write a Comprehension that returns the indices of the objects present in the bag. For that, we'll add a [conditional List Comprehension](https://riptutorial.com/python/example/767/conditional-list-comprehensions), which will [enumerate](https://www.guru99.com/python-enumerate-function.html) the content of the bag, and only filter the <code class="cm-s-ipython language-python"><span class="cm-keyword">True</span></code> values. Let's try to write this Comprehension.



In [56]:
sac = (True, False, True)

# we test the generator with a List Comprehension
#objets =[i for i in range(len(sac)) if sac[i]]
objets = [i for i, val in enumerate(sac) if val]

print(objets)

[0, 2]


We obtain the list of indices of the objects we are interested in. Now, we are going to modify this generator to create the list of weight values of the objects present in the bag.


We will directly use this method to implement a function for calculating the weight of the bag. And by the way, we also implement the calculation of the value of the bag, on the same principle.

In [57]:
def poids_contenu(sac):
 """
 This function returns the sum of the weights of the objects in the bag
 """
 return sum([poids_objets[i] for i, val in enumerate(sac) if val])

def valeur_contenu(sac):
 """
 This function returns the sum of the values of the objects in the bag
 """
 # sum of the values of the objects present in the bag
 return sum([valeur_objets[i] for i, val in enumerate(sac) if val])

Everything works correctly&nbsp;? To check this, the easiest way is to generate a random solution, and to test our two functions on this solution.

In [58]:
import random

random.seed(a=3)
nb_objets = 5
poids_objets, valeur_objets = random_objets(10, 10)

test = tuple(random.choice([False, True]) for _ in range(nb_objets))
for o in range(nb_objets):
 print("%d:(%d,%d) "%(o, poids_objets[o], valeur_objets[o]), end='')

print("")
print([i for i, val in enumerate(test) if val]) # displays the indices of the True values of the solution
print("weight=%d, valeur=%d" %(poids_contenu(test), valeur_contenu(test)))

0:(4,10) 1:(10,8) 2:(9,10) 3:(3,2) 4:(6,10) 
[1, 2]
weight=19, valeur=18


Apparently, it works&nbsp;!



# 2 simple heuristics
We can now focus on the implementation of a _Hill Climbing_. To begin with, it is necessary to implement the function for calculating the neighbourhood of a solution mentioned above.

## 2.1 Generation of neighbourhood
We therefore want to generate the set of all the neighbours of a solution (so, of a tuple of Booleans). To be sure, write this algorithm in more detail. You can consider that a function of cloning a solution is at your disposal.

<font color = "yellow">
<em>
</em>

Now, what it left is to implement this algorithm.




In [60]:
def voisinage(sac):
 """
 This function is a generator of all valid neighbours of a solution
 """
 for k in range(len(sac)):

 # a hint: we want the states of the first k-1 objects and the last |sac|-k-1 objects
 # and the inverse state of object k (pay attention to the slicing bounds)
   neighbor = sac[:k] + (not sac[k],) + sac[k+1:]
   if (poids_contenu(neighbor) <= capacite):
    yield neighbor


# we test
test = (True, False, True)
print (test, "\nneighbours :")
for voisin in voisinage(test):
 print(voisin)

(True, False, True) 
neighbours :
(False, False, True)
(True, False, False)


Does that sound good to you&nbsp;? Why aren't there more solutions&nbsp;?

<font color = "yellow">
<em>Because other combinations of booleans correspond to unfeasible solutions because they will exceed the bag's capacity</em>

## 2.2 Hill Climbing


We can finally get down to implementing our heuristic _Hill Climbing_. If we were to write the algorithm in more detail, what could we propose&nbsp;?

<em>TO BE COMPLETED</em>

What is wrong with this algorithm&nbsp;? How could it be fixed&nbsp;?

<em>TO BE COMPLETED</em>

Now let's get started with its implementation&nbsp;!
</blockquote>

In [ ]:
def hill_climbing(solution_initiale):
 """
 1. We start from an element of our search set that we state current element
 2. We consider the neighbourhood of the current element and we choose the best of them
 as the new current element
 3. We loop until convergence on a local optimum
 """

 solution_courante = solution_initiale
 nouveau = True
 nb_iter = 0 # uniquement utilisé pour l'affichage

 while (nouveau):
 nb_iter += 1

 # we go through all the neighbours of the current solution to keep the best one
 #TO BE COMPLETED
 nouveau = (meilleure_solution != solution_courante)
 solution_courante = meilleure_solution

 return solution_courante

## 2.3 Running the heuristic
Let's test this algorithm by taking an empty bag as an initial solution, and display the proposed solution&nbsp;:

In [ ]:
random.seed(a=3)
capacite = 20
nb_objets = 100
poids_objets, valeur_objets=random_objets(10, 10)

sac = (False,)*nb_objets # a useful feature in Python

print ("local optimization")
sol = hill_climbing(sac)
print("final value = " + str(valeur_contenu(sol)) + ", Capacity=" + str(poids_contenu(sol)) + "/" + str(capacite))
print([i for i, val in enumerate(sol) if val]) # liste des objets dans le sac

Three objects is not much&nbsp;! That said, maybe we can do something to improve that. Think about it&nbsp;: this algorithm is [deterministic](https://en.wikipedia.org/wiki/Deterministic_algorithm). When considering an optimization algorithm like tabu search, what does that imply in terms of solution quality for a given instance&nbsp;?

<em>TO BE COMPLETED</em>

Doesn't that give you an idea to improve the result produced by the algorithm&nbsp;? How could one implement such an approach&nbsp;?

<em>TO BE COMPLETED</em>

It's simple to set up, especially in Python. Which function should you implement&nbsp;? What exactly will she do&nbsp;? A hint, Python’s random generation functions, e.g. [<code class="cm-s-ipython language-python"><span class="cm-builtin">random_choice</span></code></a>](https://docs.python.org/3/library/random.html#random.choice) should be helpful.

<em>TO BE COMPLETED</em>

Let’s go&nbsp;!



In [ ]:
def random_solution():
 sac = tuple( #TO BE COMPLETED

 while (poids_contenu(sac) > capacite):
 # list of `True` value indices in the bag
 objets_presents = tuple(
 #TO BE COMPLETED

 # random choice of an object to delete among those present in the bag
 objet_supprime = #TO BE COMPLETED

 # deletion of the randomly selected object
 sac = #TO BE COMPLETED

 return sac

Now we just have to test all this. To start, let's try with 10 iterations. At each iteration, display the search result (like the final display l.14 of the code to be completed, but without displaying the detail of the solution), so that we can see a little what is happening pass.

In [ ]:
val_max = 0
for _ in range(5):
 #TO BE COMPLETED

print("\nfinal value = %d, Capacity=%d/%d\n" % (val_max, poids_contenu(sol_max),capacite))
print([i for i, val in enumerate(sol_max) if val])

We have significantly improved the result&nbsp;! And we can see that the local optimum found depends on the initial solution. By testing several initial solutions, one improves in certain cases the solution found. What is this technique called&nbsp;?

<em>TO BE COMPLETED</em>

<em>TO BE COMPLETED</em>

#3 Metaheuristics
Finally, we have already implemented a first metaheuristic, but we are not going to stop there&nbsp;!

## 3.1 Search with a tabu list
A metaheuristic a little smarter than a Hill Climbing should give best results. Among all the metaheuristics in the literature, the [tabu list search](https://en.wikipedia.org/wiki/tabu_search) method (often called _tabu method_ by shortcuts) is the one that could be adapted most easily based on the Hill Climbing code. Why&nbsp;?

<em>TO BE COMPLETED</em>

What algorithmic data structure do we need to handle this&nbsp;?

<em>TO BE COMPLETED</em>

To implement this in Python, which structure do you think is suitable from a performance point of view&nbsp;?

<em>TO BE COMPLETED</em>

This time, we have everything we need. So let's get started with this implementation&nbsp;:

In [ ]:
from collections import deque

def recherche_tabou(solution_initiale, taille_tabou, iter_max):
 """
 1. We start from an element of our search set that we state current element
 2. We consider the neighbourhood of the current element, we choose the best of them
 as a new current element, among those missing from the tabu list, and we add it
 to the tabu list
 3. We run a loop until the output condition is reached.
 """
 nb_iter = 0
 liste_tabou = #TO BE COMPLETED

 # variables solutions for finding the neighbour optimal not tabu
 solution_courante = solution_initiale
 meilleure = solution_initiale
 meilleure_globale = solution_initiale

 # variables values for finding neighbour optimal not tabu
 valeur_meilleure = valeur_contenu(solution_initiale)
 valeur_meilleure_globale = valeur_meilleure


 while (nb_iter < iter_max):
 valeur_meilleure = -1

 # we go through all the neighbours of the current solution
 for voisin in voisinage(solution_courante):
 #TO BE COMPLETED

 # we update the best solution encountered since the beginning
 if valeur_meilleure > valeur_meilleure_globale:
 meilleure_globale = meilleure
 valeur_meilleure_globale = valeur_meilleure
 nb_iter = 0
 else:
 nb_iter += 1

 # we go to the best non-tabu neighbour found
 solution_courante = meilleure

 # we update the tabu list
 #TO BE COMPLETED

 return meilleure_globale

A test with a tabu list of size 5 on the same instance as before will allow us to compare the result with that obtained by local optimization&nbsp;:

In [ ]:
nb_objets = 100
capacite = 20
random.seed(a=3)
poids_objets, valeur_objets = random_objets(10, 10)
sac = (False,)*nb_objets
taille_tabou=5
iter_max=30

print("tabu of size 5")
sol=recherche_tabou(sac, taille_tabou, iter_max)
print("final value = " + str(valeur_contenu(sol)) + ", Capacity="+str(poids_contenu(sol)) + "/" + str(capacite))

We have significantly improved the quality compared to the solution obtained simply by local optimization&nbsp;! That is to say if this first approach was ineffective… We are even better than with the multi-start&nbsp;!

## 3.2 Tabu with multi-start

Moreover, we can reuse this principle of multi-start. Implement it by strengthening diversification (500 iterations, for example)&nbsp;:

In [ ]:
random.seed(a=3)

sol_max = None
#TO BE COMPLETED
print("final value = " + str(valeur_contenu(sol_max)))

The solution is further improved. That said, the overhead in computation time is substantial.


However, there is a way to speed up the calculations without touching much. Some of the functions we have implemented are _pure_. And that will allow us to set up an acceleration of the execution of the code. Do you have any idea how to do it&nbsp;?

<em>TO BE COMPLETED</em>

Moreover, fortunately we chose a tuple to represent a solution, because we could not have implemented this technique with lists (which are not hashable).

First of all, let's restart the calculation by counting the time CPU, to get an idea of the gain that will be obtained afterwards.



In [ ]:
import time
random.seed(a=3)

sol_max = None
val_max = 0

start = time.process_time()

# multi-start of 500 iterations
#TO BE COMPLETED

stop = time.process_time()

print("final value = " + str(valeur_contenu(sol_max)))
print("calculated in ", stop-start, 's')

Now let's test this technique, by redefining the functions involved.

In [ ]:
from functools import lru_cache

#TO BE COMPLETED

And let's test this by rerunning the previous calculation&nbsp;:

In [ ]:
random.seed(a=3)

sol_max = None
val_max = -1

start = time.process_time()
# multi-start of 500 iterations
#TO BE COMPLETED

stop = time.process_time()

print("final value = " + str(valeur_contenu(sol_max)))
print("calculated in ", stop-start, 's')

The acceleration is impressive, the computation time has been almost divided by 4&nbsp;! And the more calculations we make on this instance, the more we will accelerate (in exchange for a higher memory consumption). To convince you of this, just run the same calculation again, but with a different seed&nbsp;:

In [ ]:
#### Copy-paste the code from the previous cell
random.seed(a=5)

sol_max = None
val_max = 0

start = time.process_time()
# multi-start of 500 iterations
#TO BE COMPLETED

stop = time.process_time()

print("final value = " + str(valeur_contenu(sol_max)))
print("calculated in ", stop-start, 's')

There too, we went faster, while the execution scenario is not the same, which means that the initial solutions considered are different.


And if you're wondering why we still get the same solution value, we'll have the answer at the next Workshop&nbsp;!

# 4. Conclusion

This Workshop is now over, and we have finally managed to do something of this knapsack problem, since you have implemented, not one, but three metaheuristics, a Hill-Climbing with multi-start, a simple tabu search, and one with multi-start.


Even if you start with a different implementation for your project, you can see that it is not that complicated. Take the case of a genetic algorithm. In the case of a knapsack, what would a gene represent&nbsp;?

<em>TO BE COMPLETED</em>

If we had to implement, we would already have a lot of reusable functions (random solution generation, assessment of the feasibility of a solution, of its value). It would just be necessary to implement a crossing function and a mutation function. What kind of crossover could be considered&nbsp;?

<em>TO BE COMPLETED</em>

What about the mutation&nbsp;?

<em>TO BE COMPLETED</em>

Once these additional functions are implemented, the body of the algorithm will not be much more complicated than that of the tabu algorithm that we have just written.


But the question that arises is&nbsp;: are the solutions we build of good quality&nbsp;? Ultimately, we don't know.

It will be necessary to analyse the behaviour of the algorithm according to certain parameters, to be able to determine if it is effective or not, and if necessary to optimize its operation. And that’s the subject of the next Workshop&nbsp;!